#installing dependencies

# New Section

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80
Mon Mar 28 20:50:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                           

In [ ]:
!pip install optuna==2.3.0
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert
!wget https://huggingface.co/UBC-NLP/MARBERT/resolve/main/MARBERT_pytorch_verison.tar.gz
!tar -xvf MARBERT_pytorch_verison.tar.gz
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_train.tsv
!wget https://raw.githubusercontent.com/UBC-NLP/marbert/main/examples/UBC_AJGT_final_shuffled_test.tsv
!mkdir -p AJGT
!mv UBC_AJGT_final_shuffled_train.tsv ./AJGT/UBC_AJGT_final_shuffled_train.tsv
!mv UBC_AJGT_final_shuffled_test.tsv ./AJGT/UBC_AJGT_final_shuffled_test.tsv
!pip install GPUtil pytorch_pretrained_bert transformers

     |████████████████████████████████| 258 kB 5.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 210 kB 45.4 MB/s 
     |████████████████████████████████| 81 kB 7.5 MB/s 
     |████████████████████████████████| 78 kB 7.0 MB/s 
     |████████████████████████████████| 150 kB 50.9 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
     |████████████████████████████████| 113 kB 45.4 MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-py3-none-any.whl size=359772 sha256=9326df2a68b5231b2bfe7875597c257102a48610d3bf3492058e0b918d3d8b9c
  Stored in directory: /root/.cache/pip/wheels/38/61/9e/955ab1890f6cab231b1d756db63f36c711968a324296e0b649
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=3ef5f55013ef2c04d0db1031123ecd0b9fc6150139955431942781d032eae66e
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f

In [ ]:
!mkdir data
!mkdir train

#Creating training datasets

In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split
all_datasets= []

In [ ]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [ ]:
DATA_COLUMN = "Tweets"
LABEL_COLUMN = "labels"

##HARD - Balanced

In [ ]:
df_train=pd.read_csv("/content/train.csv")
df_valid=pd.read_csv("/content/dev.csv")

In [ ]:
df_valid

,Tweets,labels
0,افطرت عليك بعقاء واثنين من فروخها الجن,NOT_HS
1,داليا مبارك مادري ليش تقرفت,NOT_HS
2,ابديت السناب الجديد حاس الناس حوس أشوف مشاهير ...,NOT_HS
3,هييه والله وااايدد,NOT_HS
4,اكيد اخس شي,NOT_HS
...,...,...
1265,روما محظوظين بذا المدرب بيروتي يسحب في رجوله ا...,NOT_HS
1266,هلا والله بالحب هلا لولو من جد مو طايقتهم قللل...,NOT_HS
1267,رينز فاز يعنني اوه شوفو العرض الأسطوري تفو علي...,NOT_HS
1268,ييييع والله شيء يلوع الكبد مريضات الله يشفيهم,NOT_HS


In [ ]:
df_train[LABEL_COLUMN]=df_train[LABEL_COLUMN].map({'NOT_HS':0,'HS':1})
df_valid[LABEL_COLUMN]=df_valid[LABEL_COLUMN].map({'NOT_HS':0,'HS':1})

In [ ]:
# train_HARD, test_HARD = train_test_split(df_HARD, test_size=0.2, random_state=42)
label_list_HARD = [0,1]
data_Hard = Dataset("HARD", df_train, df_valid, label_list_HARD)
all_datasets.append(data_Hard)

#Trainer

In [ ]:
from arabert.preprocess import ArabertPreprocessor
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import optuna

In [ ]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [ ]:
for x in all_datasets:
  print(x.name)

HARD


You can choose which model, and dataset from here along with the max sentence length

In [ ]:
dataset_name = 'HARD'
model_name = 'UBC-NLP/MARBERT'
task_name = 'classification'
max_len = 256

In [ ]:
for d in all_datasets:
  if d.name==dataset_name:
    selected_dataset = d
    print('Dataset found')
    break

Dataset found


In [ ]:

arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

selected_dataset.train[DATA_COLUMN] = selected_dataset.train[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))
selected_dataset.test[DATA_COLUMN] = selected_dataset.test[DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))

In [ ]:
test_set =pd.read_csv("/content/Test.csv")["Tweets"].apply(lambda x:   arabert_prep.preprocess(x))
len(test_set)

2541

In [ ]:
selected_dataset.test[DATA_COLUMN]

0                  افطرت عليك بعقاء واثنين من فروخها الجن
1                             داليا مبارك مادري ليش تقرفت
2       ابديت السناب الجديد حاس الناس حوس أشوف مشاهير ...
3                                       هييه والله واايدد
4                                             اكيد اخس شي
                              ...                        
1265    روما محظوظين بذا المدرب بيروتي يسحب في رجوله ا...
1266    هلا والله بالحب هلا لولو من جد مو طايقتهم قللع...
1267    رينز فاز يعنني اوه شوفو العرض الأسطوري تفو علي...
1268          ييع والله شيء يلوع الكبد مريضات الله يشفيهم
1269              تحسينها ع كليجه م اكلت شي واضح من الصوت
Name: Tweets, Length: 1270, dtype: object

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [ ]:
class embDataset(Dataset):
    def __init__(self, text, model_name, max_len):
      super(BERTDataset).__init__()
      self.text = text
      # self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      # self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [ ]:
# test_set= embDataset()
len(test_set)

2541

In [ ]:
label_map = { v:index for index, v in enumerate(selected_dataset.label_list) }
print(label_map)
train_dataset = BERTDataset(selected_dataset.train[DATA_COLUMN].to_list(),selected_dataset.train[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
valid_dataset = BERTDataset(selected_dataset.test[DATA_COLUMN].to_list(),selected_dataset.test[LABEL_COLUMN].to_list(),model_name,max_len,label_map)


{0: 0, 1: 1}


Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

In [ ]:
test_set = embDataset(test_set,model_name,max_len)
len(valid_dataset)

1270

In [ ]:
print(test_set)

In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:

Predictions = []
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  Predictions.append(preds)
  #print(classification_report(p.label_ids,preds))
  #print(confusion_matrix(p.label_ids,preds))
  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

# HyperParameter Search

you can change the batch size and gradient accumulation from here



In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 2e-8
training_args.lr_scheduler_type = 'cosine'
training_args.fp16 = True
training_args.per_device_train_batch_size = 8
training_args.per_device_eval_batch_size = 8
training_args.gradient_accumulation_steps = 4
training_args.num_train_epochs= 8
training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000
# training_args.save_steps = 
#training_args.eval_steps = 
training_args.disable_tqdm = True
# print("Logging Step:", training_args.logging_steps)
# print("Eval Step:",training_args.eval_steps)

In [ ]:
steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)

277
2216


In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=test_dataset, 
    model_init=model_init,
    compute_metrics=compute_metrics,
)

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

here you can define your search space.

the `my_hp_space` function defines the hyper parameter super set, of which you can choose a subset (or even the whole set) for the grid search


Note: You can include the opch count as a hyperparameter, but this will drasticly increase the search space, I prefer setting a fixed epcoh size, then I manually search for the highest score between the epochs since optuna can't do that as far as I know

In [ ]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-5, 7e-5, step=1e-5),
        "seed": trial.suggest_categorical("seed", [0, 1, 42, 666, 123, 12345]),
        "warmup_steps": trial.suggest_int("warmup_steps",0,total_steps*0.1,step=total_steps*0.1*0.5)
    }

search_space = {
    "learning_rate":  list(np.arange(2e-5, 7e-5, 1e-5)),
    "seed":  [0, 1, 42, 666, 123, 12345],
    "warmup_steps": list(range(0, int((total_steps)*0.1)+1, int(total_steps*0.1*0.5)))
}
search_space

{'learning_rate': [2e-05,
  3.0000000000000004e-05,
  4.000000000000001e-05,
  5.000000000000001e-05,
  6.000000000000001e-05],
 'seed': [0, 1, 42, 666, 123, 12345],
 'warmup_steps': [0, 3, 6]}

In [ ]:
def my_objective(metrics):
    return metrics['eval_macro_f1']

choose a study name to save it on disk

In [ ]:
name = "sa-arabert-base-v2"

In [ ]:
best_run = trainer.hyperparameter_search(direction="maximize",
                                         hp_space=my_hp_space,
                                         compute_objective=my_objective,
                                         n_trials=None,
                                         pruner=optuna.pruners.NopPruner(),
                                         sampler=optuna.samplers.GridSampler(search_space),
                                         study_name=name,
                                         storage="sqlite:////content/drive/MyDrive/{}.db".format(name),
                                         load_if_exists=False # you can change this to true, for continuing the search
                                         )

RuntimeError: ignored

In [ ]:
best_run

#Regular Training

This paert allows you to do a regular training with no hyper parameter optimization

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 2e-8
training_args.learning_rate =1e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 4


steps_per_epoch = (len(selected_dataset.train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 20
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

277
1108


In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
1,No log,0.140026,0.794059,0.794059,0.806197,0.783092,0.937795,102.058700,12.444000
2,0.220600,0.139614,0.827243,0.827243,0.822022,0.832689,0.944882,102.069600,12.442000
3,0.220600,0.202723,0.818918,0.818918,0.794516,0.849596,0.937795,102.048800,12.445000


Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
1,No log,0.140026,0.794059,0.794059,0.806197,0.783092,0.937795,102.058700,12.444000
2,0.220600,0.139614,0.827243,0.827243,0.822022,0.832689,0.944882,102.069600,12.442000
3,0.220600,0.202723,0.818918,0.818918,0.794516,0.849596,0.937795,102.048800,12.445000
4,0.078000,0.208702,0.823113,0.823113,0.808824,0.839280,0.941732,102.036900,12.446000


TrainOutput(global_step=1112, training_loss=0.13891053285530144, metrics={'train_runtime': 8938.1269, 'train_samples_per_second': 0.124, 'total_flos': 8891502749700096, 'epoch': 4.0})

In [ ]:
result=trainer.predict(test_set)


In [ ]:
print(result[0])

[[ 3.488 -3.326]
 [ 3.49  -3.264]
 [ 3.492 -3.248]
 ...
 [ 3.438 -3.318]
 [ 2.865 -2.729]
 [ 2.613 -2.348]]


In [ ]:
trainer.save_model("HS_Not_HS")

In [ ]:
def conv2txt(labels):
  txtLabels = []
  for i in labels:
    val, idx = max((val, idx) for (idx, val) in enumerate(i))
    if idx == 0:
      txtLabels.append("NOT_HS")
    elif idx == 1:
      txtLabels.append("HS")
  return txtLabels

In [ ]:
labels= conv2txt(result[0])

In [ ]:
print(labels)

['NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'HS', 'NOT_HS', 'NOT_HS', 'HS', 'HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', 'NOT_HS', '

In [ ]:
from google.colab import files
with open("labelsHS_TaskB.txt", 'w') as fd:
      for lbl in labels:
        fd.write(lbl + "\n")
files.download('labelsHS_TaskB.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
! zip -r /content/OFF_Not_OFF.zip /content/OFF_Not_OFF

  adding: content/OFF_Not_OFF/ (stored 0%)
  adding: content/OFF_Not_OFF/training_args.bin (deflated 44%)
  adding: content/OFF_Not_OFF/config.json (deflated 46%)
  adding: content/OFF_Not_OFF/pytorch_model.bin (deflated 7%)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
shutil.move('/content/OFF_Not_OFF.zip','/content/drive/MyDrive/')

Error: ignored

In [ ]:
import shutil
shutil.copy('/content/drive/MyDrive/OFF_Not_OFF.zip','/content')

'/content/OFF_Not_OFF.zip'

In [ ]:
from zipfile import ZipFile
with ZipFile('OFF_Not_OFF.zip', 'r') as zip:
	# printing all the contents of the zip file
	zip.printdir()

	# extracting all the files
	print('Extracting all the files now...')
	zip.extractall()
	print('Done!')

File Name                                             Modified             Size
content/OFF_Not_OFF/                           2022-03-25 21:24:36            0
content/OFF_Not_OFF/training_args.bin          2022-03-25 21:24:36         1967
content/OFF_Not_OFF/config.json                2022-03-25 21:24:34          633
content/OFF_Not_OFF/pytorch_model.bin          2022-03-25 21:24:36    540863661
Extracting all the files now...
Done!


In [ ]:
!unzip '/content/drive/MyDrive/OFF_Not_OFF.zip'

Archive:  /content/drive/MyDrive/OFF_Not_OFF.zip
replace content/OFF_Not_OFF/training_args.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: content/OFF_Not_OFF/training_args.bin  
replace content/OFF_Not_OFF/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: content/OFF_Not_OFF/config.json  
replace content/OFF_Not_OFF/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: content/OFF_Not_OFF/pytorch_model.bin  y



In [ ]:
import torch 
modell= torch.load('/content/content/rename/pytorch_model.bin')
print(modell)

<built-in method values of collections.OrderedDict object at 0x7fed541bd200>


In [ ]:
modell.load_state_dict(modell)‏

SyntaxError: ignored

In [ ]:
trainer = Trainer(
    model = modell(),
    args = '/content/content/rename/training_args.bin',
    train_dataset = train_dataset,
    eval_dataset='/content/Test.csv',
    compute_metrics=compute_metrics,
)

TypeError: ignored